In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, Lasso, ElasticNet
from sklearn.model_selection import train_test_split


## INTRO TO SKLEARN

In [ ]:

test_size = 0.2
shuffle = False 
noise_strength = 10 

# Create data
x = np.linspace(0, 10, 100)
y = x**2 + noise_strength * (np.random.random((len(x)))-0.5)

# Split data into train/dev/test
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_size, shuffle=shuffle)
x_train, x_dev, y_train, y_dev = train_test_split(x_train, y_train, test_size=test_size, shuffle=shuffle)

# Plot
fig = plt.figure()
plt.plot(x_train, y_train, 'k.')
plt.plot(x_dev, y_dev, 'b.')
plt.plot(x_test, y_test, 'r.')
plt.legend(['train', 'dev', 'test'])
plt.ylabel('y')
plt.xlabel('x')
plt.show()

The `score` quantifies the residual, defined as

$$ R^2 = \left( 1 - \frac{\sum_i (\hat y_i - y_i)^2}{\sum_i (y_i - \langle y \rangle)^2} \right)$$

where $\hat y$ is the prediction, $y$ is the true value, $\langle y \rangle$ is the average of the true values.

In [ ]:
import pdb
from sklearn.metrics import mean_squared_error

# Reshape y to (n, 1) for sklearn
Y = y.reshape(-1, 1)

def make_features(x, degree=1):
    X = np.zeros((len(x), degree+1))
    for i in range(degree+1):
        X[:, i] = x**i
    return X

def rmse(x, y, reg, degree):
    y_pred = reg.predict(make_features(x, degree=d))
    rmse_val = mean_squared_error(y, y_pred)
    return rmse_val


degree_list = [1, 2, 3, 5, 10]
for deg in degree_list:
    d = deg
    X = make_features(x, degree=d)

    # Split data into train/dev/test
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size, shuffle=shuffle)
    X_train, X_dev, Y_train, Y_dev = train_test_split(X_train, Y_train, test_size=test_size, shuffle=shuffle)

    # FIT!
    # reg = LinearRegression(fit_intercept=False)
    # reg = Lasso(fit_intercept=False)
    reg = ElasticNet(alpha=0.2, l1_ratio=0.5, fit_intercept=False)
    reg.fit(X_train, Y_train)


    rmse_dev = rmse(x_dev, y_dev, reg, d)
    rmse_train = rmse(x_train, y_train, reg, d)

    print('\n\n')
    print('degree = %d'%(d))
    print('--')
    print('validation score = %.4f'%( reg.score(X_dev, Y_dev) ))
    print('train score = %.4f'%(reg.score(X_train, Y_train)))
    print('--')
    print('validation RMSE = %.4f'%(rmse_dev))
    print('train RMSE = %.4f'%(rmse_train))
    print('--')
    print('coeffficients = ', reg.coef_)
    print('--')

    x_c = np.linspace(x[0], np.max(x_dev), 100)
    
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot(x_c, reg.predict(make_features(x_c, degree=d)), '--')
    ax.plot(x_train, y_train, '.')
    ax.plot(x_dev, y_dev, '.', ms=10)
    ax.plot(x_dev, reg.predict(make_features(x_dev, degree=d)), '.')
    
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_ylim(np.min(y_train), np.max(y_dev))
    ax.legend(['polynomial fit', 'train data', 'dev data', 'dev prediction'])
    plt.show()

In [ ]:

y_test_pred = reg.predict(make_features(x_test, degree=d))
rmse_test = mean_squared_error(y_test_pred, y_test)

print('test score = %.4f'%( reg.score(X_test, Y_test) ))
print('train score = %.4f'%(reg.score(X_train, Y_train)))
print('--')
print('test RMSE = %.4f'%(rmse_test))
print('train RMSE = %.4f'%(rmse_train))
print('--')
print('coeffficients = ', reg.coef_)
print('\n')
print('--')